In [2]:
import os
!pip install langchain_community chromadb tiktoken faiss-cpu huggingface_hub
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings  # Use Hugging Face embeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub  # Use Hugging Face Hub LLM
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:



os.environ["HUGGINGFACEHUB_API_TOKEN"] = "HUGGING_FACE_API_KEY"

# 1. Load and process documents (RAG)
with open("knowledge_base.txt", "w") as f:
    f.write("""
Quantum physics, also known as quantum mechanics, is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles.  It's a probabilistic theory, meaning it doesn't predict exact outcomes but rather the probabilities of different outcomes.  Here's a breakdown of key concepts:

**Key Concepts:**

*   **Quantization:** Energy, momentum, and other physical properties are not continuous but come in discrete packets called quanta.  Think of it like stairs, not a ramp.  A classic example is the quantization of light into photons.
*   **Wave-Particle Duality:** Particles, like electrons and photons, can exhibit both wave-like and particle-like behavior.  This is one of the most mind-bending aspects of quantum mechanics.  Electrons, for example, can diffract like waves and also collide like particles.
*   **Superposition:** A quantum system can exist in multiple states simultaneously until measured.  Imagine a coin spinning in the air – it's both heads and tails at the same time until it lands.  This is represented mathematically by a wave function.
*   **Entanglement:** Two or more quantum particles can become linked in such a way that their fates are intertwined, even when separated by large distances.  Measuring the state of one entangled particle instantaneously affects the state of the other, regardless of the distance. This "spooky action at a distance," as Einstein called it, has profound implications for quantum computing and communication.
*   **Uncertainty Principle:** There is a fundamental limit to the precision with which certain pairs of physical properties, like position and momentum, can be known simultaneously.  The more precisely you know a particle's position, the less precisely you can know its momentum, and vice versa.
*   **Quantum Field Theory:**  This is a more advanced framework that combines quantum mechanics with special relativity.  It describes particles as excitations of underlying quantum fields.

**Applications of Quantum Physics:**

*   **Quantum Computing:** Exploits quantum phenomena like superposition and entanglement to perform computations that are impossible for classical computers.  This has the potential to revolutionize fields like medicine, materials science, and artificial intelligence.
*   **Quantum Cryptography:** Uses the principles of quantum mechanics to create secure communication channels.  Entanglement, for example, can be used to detect eavesdropping.
*   **Lasers:** Based on the principle of stimulated emission of radiation, lasers have numerous applications in everything from barcode scanners to laser surgery.
*   **Medical Imaging:** Techniques like MRI (Magnetic Resonance Imaging) rely on the quantum properties of atomic nuclei.
*   **Materials Science:** Understanding quantum mechanics is crucial for designing new materials with specific properties.
*   **Semiconductors:** The foundation of modern electronics, semiconductors rely on quantum mechanical principles.

**Experimental Evidence:**

*   **Double-Slit Experiment:** A classic experiment that demonstrates wave-particle duality.  Electrons, when passed through two slits, create an interference pattern, as if they were waves, even when fired one at a time.
*   **Bell's Theorem:**  A theorem that demonstrates the reality of entanglement.  Experiments have confirmed Bell's inequalities, providing strong evidence for the "spooky action at a distance."
*   **Stern-Gerlach Experiment:**  An experiment that provided evidence for the quantization of angular momentum.

**Challenges and Open Questions:**

*   **Interpretation of Quantum Mechanics:**  There are various interpretations of quantum mechanics, such as the Copenhagen interpretation and the many-worlds interpretation, each with its own philosophical implications.
*   **Quantum Gravity:**  One of the biggest challenges in physics is to reconcile quantum mechanics with general relativity, Einstein's theory of gravity.  A theory of quantum gravity is needed to understand phenomena like black holes and the early universe.

This is a brief overview of quantum physics.  It's a vast and complex field with many fascinating aspects.  If you have any specific questions, feel free to ask!
""")

loader = TextLoader("knowledge_base.txt")
documents = loader.load()

# Use Hugging Face Embeddings
embeddings = HuggingFaceEmbeddings()
HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(documents, embeddings)

# 2. Define Pydantic output schema
class Answer(BaseModel):
    text: str = Field(description="The answer to the question.")
    sources: List[str] = Field(description="List of source document excerpts used.")

    @validator("sources")
    def sources_must_be_list(cls, sources):
        if not isinstance(sources, list):
            raise ValueError("Sources must be a list.")
        return sources

# 3. Create a RetrievalQA chain  with Hugging Face LLM
template = """Answer the question based on the context below. If the question cannot be answered using the context, say "I don't know".

Context: {context}

Question: {question}

Answer:"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])
output_parser = PydanticOutputParser(pydantic_object=Answer)

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.0,"max_length": 500})

chain = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=db.as_retriever(),
chain_type_kwargs={"prompt": prompt}
)




# question = "Describe Quantim physics in 15 words"
# result = chain.run(question)

# print(result)

questions = [
    "Explain quantum mechanics in 15 words.",
    "Summarize the key principles of quantum computing in 15 words.",
    "Describe the concept of entanglement in 15 words.",
    "What are the core ideas of quantum field theory in 15 words?",
    "Provide a brief explanation of wave-particle duality in 15 words."
]

for question in questions:
    result = chain.run(question)
    print(f"Question: {question}")
    print(f"Answer: {result}")
    print("-" * 50)



<ipython-input-28-8540c0028824>:45: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # You can specify a model here if you want: HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
<ipython-input-28-8540c0028824>:54: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("sources")


Question: Explain quantum mechanics in 15 words.
Answer: Quantum physics, also known as quantum mechanics, is a fundamental theory in physics that describes the physical properties of nature at the scale of atoms and subatomic particles.
--------------------------------------------------
Question: Summarize the key principles of quantum computing in 15 words.
Answer: I don't know
--------------------------------------------------
Question: Describe the concept of entanglement in 15 words.
Answer: Two or more quantum particles can become linked in such a way that their fates are intertwined, even when separated by large distances.
--------------------------------------------------
Question: What are the core ideas of quantum field theory in 15 words?
Answer: It describes particles as excitations of underlying quantum fields
--------------------------------------------------
Question: Provide a brief explanation of wave-particle duality in 15 words.
Answer: Electrons, for example, can di